In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1579201521534_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-1>

In [49]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from pyspark.sql.types import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
ss = SparkSession.builder.getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
chest_path = 's3://msds697-g2/Chest/'
wrist_path = 's3://msds697-g2/Wrist/'
nums = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [77]:
chest = ss.read.csv(chest_path+'S_2.csv', header=True, inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
wrist = ss.read.csv(wrist_path+'S_2.csv', header=True, inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
def clean_df(df):
    df = df.filter(df.TEMP.isNotNull())
    return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
def combine(wrist, chest):
    chest = chest.drop('_c0')
    wrist = wrist.withColumnRenamed('_c0', 'col_1')
    wrist = clean_df(wrist)
    joined = chest.join(wrist, 'col_1')
    joined = joined.filter(joined.label.isin())
    return joined

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
chest = ss.read.csv(f"{chest_path}S_2.csv", header=True, inferSchema=True)
wrist = ss.read.csv(f"{wrist_path}S_2.csv", header=True, inferSchema=True)
wrist = wrist.drop('label')
all_data = combine(wrist, chest)
all_data = all_data.filter(all_data.label.isin([1, 2, 3]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
# all_data2 = all_data.withColumn('wrist_label2', all_data.wrist_label.cast(IntegerType()))\
#                         .drop('wrist_label')\
#                             .withColumnRenamed('wrist_label2', 'wrist_label')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [65]:
# all_data2.filter(all_data2.label != all_data2.wrist_label).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
for sub_id in nums[1:]:
    chest = ss.read.csv(f"{chest_path}S_{sub_id}.csv", header=True, inferSchema=True)
    wrist = ss.read.csv(f"{wrist_path}S_{sub_id}.csv", header=True, inferSchema=True)
    wrist = wrist.drop('label')
    joined = combine(wrist, chest)
    joined = joined.filter(joined.label.isin([1, 2, 3]))
    all_data = all_data.union(joined)
all_data.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[col_1: double, label: int, sub_id: string, chest_ACC_1: double, chest_ACC_2: double, chest_ACC_3: double, chest_ECG: double, chest_EMG: double, chest_EDA: double, chest_Temp: double, chest_Resp: double, ACC_0: double, ACC_1: double, ACC_2: double, BVP: double, EDA: double, TEMP: double]

In [72]:
data_split = all_data.randomSplit([.8, .2], seed=2) 
train_data = data_split[0].cache()
test_data = data_split[1].cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [75]:
test_data

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [88]:
test_data.write.option('path', '/testdata').saveAsTable('testdata')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [89]:
all_data.write.option('path', '/alldata').saveAsTable('alldata')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [87]:
all_data.join(test_data, ['sub_id', 'col_1'], 'leftanti')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+------+-----+------------------+--------------------+-------------------+--------------------+-------------------+------------------+----------+-----------------+-----+-----+-----+------+--------+-----+
|sub_id| col_1|label|       chest_ACC_1|         chest_ACC_2|        chest_ACC_3|           chest_ECG|          chest_EMG|         chest_EDA|chest_Temp|       chest_Resp|ACC_0|ACC_1|ACC_2|   BVP|     EDA| TEMP|
+------+------+-----+------------------+--------------------+-------------------+--------------------+-------------------+------------------+----------+-----------------+-----+-----+-----+------+--------+-----+
|    S2|315.75|    1| 0.864799976348877| -0.1345999836921692|-0.3181999921798706| -0.1154022216796875| 0.0274200439453125|5.1868438720703125| 29.211609|-0.66070556640625| 43.0|-22.0| 41.0| -75.9| 1.47781|35.81|
|    S2|367.25|    1|0.6258000135421753|-0.10799998044967651|-0.7045999765396118|0.015609741210937498| 0.0077362060546875|     4.50439453125|  28.97876|  -1

In [20]:
wrist = wrist.withColumnRenamed('_c0', 'col_1')
wrist = clean_df(wrist)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…